In [2]:
import numpy as np

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

data_dir = "/kaggle/input/characters-data/dataset"

dataset = datasets.ImageFolder(root=data_dir, transform=transform)


train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset,[train_size, val_size, test_size])


In [13]:
print(dataset.class_to_idx)

{'digit_0': 0, 'digit_1': 1, 'digit_2': 2, 'digit_3': 3, 'digit_4': 4, 'digit_5': 5, 'digit_6': 6, 'digit_7': 7, 'digit_8': 8, 'digit_9': 9, 'lower_a': 10, 'lower_b': 11, 'lower_c': 12, 'lower_d': 13, 'lower_e': 14, 'lower_f': 15, 'lower_g': 16, 'lower_h': 17, 'lower_i': 18, 'lower_j': 19, 'lower_k': 20, 'lower_l': 21, 'lower_m': 22, 'lower_n': 23, 'lower_o': 24, 'lower_p': 25, 'lower_q': 26, 'lower_r': 27, 'lower_s': 28, 'lower_t': 29, 'lower_u': 30, 'lower_v': 31, 'lower_w': 32, 'lower_x': 33, 'lower_y': 34, 'lower_z': 35, 'upper_A': 36, 'upper_B': 37, 'upper_C': 38, 'upper_D': 39, 'upper_E': 40, 'upper_F': 41, 'upper_G': 42, 'upper_H': 43, 'upper_I': 44, 'upper_J': 45, 'upper_K': 46, 'upper_L': 47, 'upper_M': 48, 'upper_N': 49, 'upper_O': 50, 'upper_P': 51, 'upper_Q': 52, 'upper_R': 53, 'upper_S': 54, 'upper_T': 55, 'upper_U': 56, 'upper_V': 57, 'upper_W': 58, 'upper_X': 59, 'upper_Y': 60, 'upper_Z': 61}


In [4]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 📌 Preparing the model

In [5]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(64 * 35 * 35, 64)
        self.fc2 = nn.Linear(64, 62)

        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        
        x = torch.flatten(x, 1)

        x = self.dropout(self.fc1(x))
        x = self.fc2(x)
        return x


In [6]:
model = CNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CNN(
  (block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=78400, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=62, bias=Tru

In [7]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.7)

# 📌 Training

# 📌 Testing

In [ ]:
num_epochs = 20

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # force it to GPU
        
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # loss function
        
        optimizer.zero_grad()  # Reset gradients
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get predicted labels
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    scheduler.step()  # Adjust learning rate
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total  # Calculate training accuracy

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # Disable gradients computations
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            
            _ , predicted = torch.max(outputs, 1)  # Get predicted labels
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total  # Calculate accuracy

    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch+1:2d}/{num_epochs:2d}] |   "
          f"Train Loss: {epoch_loss:8.4f}, Train Acc: {epoch_acc*100:5.2f}%   |   "
          f"Val Loss: {val_loss:8.4f}, Val Acc: {val_accuracy:5.2f}%")


Epoch [ 1/20] |   Train Loss:   1.8722, Train Acc: 62.66%   |   Val Loss:   0.5212, Val Acc: 87.48%
Epoch [ 2/20] |   Train Loss:   0.4240, Train Acc: 88.49%   |   Val Loss:   0.3208, Val Acc: 91.77%
Epoch [ 3/20] |   Train Loss:   0.2370, Train Acc: 93.00%   |   Val Loss:   0.3491, Val Acc: 91.16%
Epoch [ 4/20] |   Train Loss:   0.1601, Train Acc: 95.09%   |   Val Loss:   0.3124, Val Acc: 91.99%
Epoch [ 5/20] |   Train Loss:   0.1251, Train Acc: 96.12%   |   Val Loss:   0.2321, Val Acc: 94.67%
Epoch [ 6/20] |   Train Loss:   0.0693, Train Acc: 97.79%   |   Val Loss:   0.2263, Val Acc: 95.01%
Epoch [ 7/20] |   Train Loss:   0.0612, Train Acc: 98.04%   |   Val Loss:   0.2112, Val Acc: 95.39%
Epoch [ 8/20] |   Train Loss:   0.0503, Train Acc: 98.28%   |   Val Loss:   0.6678, Val Acc: 86.86%
Epoch [ 9/20] |   Train Loss:   0.0436, Train Acc: 98.63%   |   Val Loss:   0.2176, Val Acc: 95.41%
Epoch [10/20] |   Train Loss:   0.0408, Train Acc: 98.64%   |   Val Loss:   0.2022, Val Acc: 96.09%


In [9]:
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * test_correct / test_total

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Test Loss: 0.2763, Test Accuracy: 96.85%


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

loss_df = pd.DataFrame({
    "Epoch": list(range(1, num_epochs + 1)) * 2,
    "Loss": train_losses + val_losses,
    "Type": ["Train"] * num_epochs + ["Validation"] * num_epochs
})

sns.set(style="whitegrid")
sns.lineplot(data=loss_df, x="Epoch", y="Loss", hue="Type", marker="o")

plt.xticks(range(1, num_epochs + 1, 2))
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(title="Loss Type")
plt.show()

In [11]:
torch.save(model.state_dict(), "model.pt")